## Parte 1
Encontrar la localización más baja que se corresponda con alguna de las semillas iniciales

In [1]:
import re

In [88]:
def get_input(filename):
    line2state = {'seed-to-soil map:': 's2s',
                  'soil-to-fertilizer map:': 's2f',
                  'fertilizer-to-water map:': 'f2w',
                  'water-to-light map:': 'w2l',
                  'light-to-temperature map:': 'l2t',
                  'temperature-to-humidity map:': 't2h',
                  'humidity-to-location map:': 'h2l'}
    state = 'init'
    conversion_map = {}
    
    with open(filename) as file:
        for i, line in enumerate(file):
            if i == 0:
                seed_line = line
                parts = seed_line.split(':')
                seed_ids = parts[1].split()
                seed_ids = [int(s) for s in seed_ids]
                continue
            
            if line.strip() == '':
                continue
                
            new_state = line2state.get(line.strip(), 'no')
            if new_state != 'no':
                state = new_state
                conversion_map[state] = {}
                continue
            
            
            nums_str = re.findall('\d+', line)
            destination_start = int(nums_str[0])
            source_start = int(nums_str[1])
            range_length = int(nums_str[2])
            
            source_nums = (source_start, source_start + range_length - 1)
            destination_nums = (destination_start, destination_start + range_length - 1)
            conversion_map[state][source_nums] = destination_nums
        
    return conversion_map, seed_ids

In [89]:
def traducir_rango(rango_orig, mapa_key, mapa_val):
    traducido = []
    restos = []
    
    if rango_orig[1] < mapa_key[0] or rango_orig[0] > mapa_key[1]:
        restos.append(rango_orig)
    else:
        
        # orig:  <-------------------------->......
        # mapa:  ......<-------------------------->
        # resul: <----><-------------------->......
        if rango_orig[0] < mapa_key[0]:
            restos.append((rango_orig[0], mapa_key[0] - 1))
            rango_orig = (mapa_key[0], rango_orig[1])
        
        # orig:  <-------------------------------->
        # mapa:  <---------------------->..........
        # resul: <----------------------><-------->
        if rango_orig[1] > mapa_key[1]:
            restos.append((mapa_key[1] + 1, rango_orig[1]))
            rango_orig = (rango_orig[0], mapa_key[1])
            
        offset = mapa_val[0] - mapa_key[0]
        traducido.append((rango_orig[0] + offset, rango_orig[1] + offset))
        
    return traducido, restos

In [90]:
def overlaps(rango_orig, mapa):
    result = False
    for m in mapa.keys():
        if rango_orig[1] >= m[0] and rango_orig[0] <= m[1]:
            return True
    return result

In [91]:
def process_level(rango_orig, conversion_map, level):    
    resultado = []
    
    acum_restos = [rango_orig]    
    while len(acum_restos) > 0:
        for mapa_key, mapa_val in conversion_map[level].items():
            if len(acum_restos) == 0:
                break
            actual = acum_restos.pop()
            
            traducido, restos = traducir_rango(actual, mapa_key, mapa_val)
            resultado += traducido
            
            for r in restos:
                if overlaps(r, conversion_map[level]):
                    acum_restos.append(r)
                else:
                    # si no hay mapa el resultado es el mismo numero
                    resultado.append(r)
                    
    return resultado

In [92]:
def process_all_level(all_ranges, level):
    processed = []
    for cur_range in all_ranges:
        result = process_level(cur_range, conversion_map, level)
        processed += result
    return processed

In [93]:
def process_full(initial_ranges):
    to_process = initial_ranges
    for level in conversion_map.keys():
        to_process = process_all_level(to_process, level)
    return to_process

In [95]:
conversion_map, seed_ids = get_input('test.txt')
seed_ranges = [(s, s) for s in seed_ids]
lista_resultado = process_full(seed_ranges)

In [97]:
min(lista_resultado, key=lambda x: x[0])

(35, 35)

In [99]:
conversion_map, seed_ids = get_input('input.txt')
seed_ranges = [(s, s) for s in seed_ids]
lista_resultado = process_full(seed_ranges)
min(lista_resultado, key=lambda l: l[0])

(993500720, 993500720)

## Parte 2:
la línea de semillas realmente describe rangos de semillas siendo 79 14 realmente el rango que empieza en 79 y tiene longitud 14.

hay que encontrar la localización mas baja correspondiente a las semillas iniciales

In [103]:
conversion_map, seed_ids = get_input('test.txt')
seed_ranges = []
for i in range(0, len(seed_ids), 2):
    seed_ranges.append((seed_ids[i], seed_ids[i] + seed_ids[i + 1] - 1))
lista_resultado = process_full(seed_ranges)

min(lista_resultado, key=lambda x: x[0])

(46, 55)

In [104]:
conversion_map, seed_ids = get_input('input.txt')
seed_ranges = []
for i in range(0, len(seed_ids), 2):
    seed_ranges.append((seed_ids[i], seed_ids[i] + seed_ids[i + 1] - 1))
lista_resultado = process_full(seed_ranges)

min(lista_resultado, key=lambda x: x[0])

(4917124, 17616610)